In [1]:
# --- LangChain and LLM Imports ---
from langchain_openai import ChatOpenAI 
from langchain_groq import ChatGroq

# --- Document Loading and Vector Store ---
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings 

# --- Prompting and Document Utilities ---
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain

# --- Core and Output Parsers ---
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables.graph import MermaidDrawMethod

# --- LangGraph for Workflow Graphs ---
from langgraph.graph import END, StateGraph

# --- Standard Library Imports ---
from time import monotonic
from dotenv import load_dotenv
from pprint import pprint
import os

# --- Datasets and Typing ---
from datasets import Dataset
from typing_extensions import TypedDict
from IPython.display import display, Image
from typing import List, TypedDict
from ragas import evaluate
from ragas.metrics import (
    answer_correctness,
    faithfulness,
    answer_relevancy,
    context_recall,
    answer_similarity
)
from pathlib import Path

import langgraph



# --- Load environment variables (e.g., API keys) ---
load_dotenv(dotenv_path=Path().resolve() / ".env", override=True)


e:\RAGAS\venv\Lib\site-packages\IPython\core\interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
e:\RAGAS\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [12]:
# Set the OpenAI API key from environment variable (for use by OpenAI LLMs)
os.environ["GOOGLE_API_KEY"] = os.getenv('GOOGLE_API_KEY')

# Retrieve the Groq API key from environment variable (for use by Groq LLMs)
groq_api_key = os.getenv('GROQ_API_KEY')

In [39]:
# Define the path to the Harry Potter PDF file.
# This variable will be used throughout the notebook for loading and processing the book.
hp_pdf_path =r"C:\Users\NAFEES J\Downloads\Harry Potter - Book 1 - The Sorcerers Stone.pdf"

In [8]:
from helper_functions import split_into_chapters, replace_t_with_space
# --- Split the PDF into chapters and preprocess the text ---

# 1. Split the PDF into chapters using the provided helper function.
#    This function takes the path to the PDF and returns a list of Document objects, each representing a chapter.
chapters = split_into_chapters(hp_pdf_path)

# 2. Clean up the text in each chapter by replacing unwanted characters (e.g., '\t') with spaces.
#    This ensures the text is consistent and easier to process downstream.
chapters = replace_t_with_space(chapters)

# 3. Print the number of chapters extracted to verify the result.
print(len(chapters))

a
17


In [18]:
import importlib
import helper_functions
importlib.reload(helper_functions)

# --- Load and Preprocess the PDF, then Extract Quotes ---

# 1. Load the PDF
loader = PyPDFLoader(hp_pdf_path)
document = loader.load()

# 2. Clean the document (remove tabs)
document_cleaned = helper_functions.replace_t_with_space(document)
print("📄 document_cleaned length:", len(document_cleaned))

# 3. Extract quotes as Documents
book_quotes_list = helper_functions.extract_book_quotes_as_documents(document_cleaned)

print("✅ Quotes Extracted:", len(book_quotes_list))

a
📄 document_cleaned length: 221
abc
Found 2 quotes on one doc
Found 10 quotes on one doc
Found 9 quotes on one doc
Found 13 quotes on one doc
Found 10 quotes on one doc
Found 9 quotes on one doc
Found 7 quotes on one doc
Found 3 quotes on one doc
Found 2 quotes on one doc
Found 8 quotes on one doc
Found 8 quotes on one doc
Found 3 quotes on one doc
Found 1 quotes on one doc
Found 6 quotes on one doc
Found 3 quotes on one doc
Found 5 quotes on one doc
Found 5 quotes on one doc
Found 10 quotes on one doc
Found 2 quotes on one doc
Found 4 quotes on one doc
Found 4 quotes on one doc
Found 4 quotes on one doc
Found 3 quotes on one doc
Found 5 quotes on one doc
Found 5 quotes on one doc
Found 11 quotes on one doc
Found 5 quotes on one doc
Found 7 quotes on one doc
Found 9 quotes on one doc
Found 6 quotes on one doc
Found 4 quotes on one doc
Found 10 quotes on one doc
Found 2 quotes on one doc
Found 5 quotes on one doc
Found 9 quotes on one doc
Found 10 quotes on one doc
Found 3 quotes on on

In [19]:
# --- Summarization Prompt Template for LLM-based Summarization ---

# Define the template string for summarization.
# This template instructs the language model to write an extensive summary of the provided text.
summarization_prompt_template = """Write an extensive summary of the following:

{text}

SUMMARY:"""

# Create a PromptTemplate object using the template string.
# The input variable "text" will be replaced with the content to summarize.
summarization_prompt = PromptTemplate(
    template=summarization_prompt_template,
    input_variables=["text"]
)

In [50]:
from langchain_google_genai import ChatGoogleGenerativeAI
importlib.reload(helper_functions)


# Gemini doesn't have an official tokenizer, so we use a rough heuristic
def num_tokens_from_string(string: str, model_name: str) -> int:
    return len(string) // 4  # Rough estimate: 4 characters per token

def create_chapter_summary(chapter):
    """
    Creates a summary of a chapter using a large language model (LLM).

    Args:
        chapter: A Document object representing the chapter to summarize.

    Returns:
        A Document object containing the summary of the chapter.
    """

    # Extract the text content from the chapter
    chapter_txt = chapter.page_content

    # Specify the LLM model and configuration
    model_name = "gemini-2.5-pro"
    llm = ChatGoogleGenerativeAI(
        model=model_name,
        temperature=0,
        convert_system_message_to_human=True
    )
    gpt_35_turbo_max_tokens = 16000  # Keep your original logic/variable name
    verbose = False  # Set to True for more detailed output

    # Calculate the number of tokens in the chapter text
    num_tokens = num_tokens_from_string(chapter_txt, model_name)

    # Choose the summarization chain type based on token count
    if num_tokens < gpt_35_turbo_max_tokens:
        # For shorter chapters, use the "stuff" chain type
        chain = load_summarize_chain(
            llm,
            chain_type="stuff",
            prompt=summarization_prompt,
            verbose=verbose
        )
    else:
        # For longer chapters, use the "map_reduce" chain type
        chain = load_summarize_chain(
            llm,
            chain_type="map_reduce",
            map_prompt=summarization_prompt,
            combine_prompt=summarization_prompt,
            verbose=verbose
        )

    # Start timer to measure summarizatime
    start_time = monotonic()

    # Create a Document object for the chapter
    doc_chapter = Document(page_content=chapter_txt)

    # Generate the summary using the selected chain
    summary_result = chain.invoke([doc_chapter])

    # Print chain type and execution time for reference
    print(f"Chain type: {chain.__class__.__name__}")
    print(f"Run time: {monotonic() - start_time}")

    # Clean up the summary text (remove double newlines, etc.)
    summary_text = helper_functions.replace_double_lines_with_one_line(summary_result["output_text"])

    # Create a Document object for the summary, preserving chapter metadata
    doc_summary = Document(page_content=summary_text, metadata=chapter.metadata)

    return doc_summary


In [41]:
import pickle
from pathlib import Path

# Where to save the checkpoint
checkpoint_path = Path("chapter_summaries_checkpoint.pkl")

# Load checkpoint if exists
if checkpoint_path.exists():
    with open(checkpoint_path, "rb") as f:
        chapter_summaries = pickle.load(f)
    start_index = len(chapter_summaries)
else:
    chapter_summaries = []
    start_index = 0

# Set how many chapters to process per run
max_chapters_per_run = 10
total_chapters = 17  # or len(chapters), but you specified 1–17
end_index = min(start_index + max_chapters_per_run, total_chapters)

# Process chapters from start_index to end_index - 1
for i in range(start_index, end_index):
    chapter = chapters[i]
    print(f"Processing chapter {i+1} of {total_chapters}")
    
    try:
        summary = create_chapter_summary(chapter)
        chapter_summaries.append(summary)
    except Exception as e:
        print(f"Error processing chapter {i+1}: {e}")
        break  # or continue to skip errors

    # Save checkpoint after each chapter for safety
    with open(checkpoint_path, "wb") as f:
        pickle.dump(chapter_summaries, f)
    print(f"Checkpoint saved at chapter {i+1}")


Processing chapter 14 of 17


e:\RAGAS\venv\Lib\site-packages\langchain_google_genai\chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Chain type: StuffDocumentsChain
Run time: 38.26162130001467
Checkpoint saved at chapter 14
Processing chapter 15 of 17


e:\RAGAS\venv\Lib\site-packages\langchain_google_genai\chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Chain type: StuffDocumentsChain
Run time: 42.91590920003364
Checkpoint saved at chapter 15
Processing chapter 16 of 17


e:\RAGAS\venv\Lib\site-packages\langchain_google_genai\chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Chain type: StuffDocumentsChain
Run time: 46.75584999995772
Checkpoint saved at chapter 16
Processing chapter 17 of 17


e:\RAGAS\venv\Lib\site-packages\langchain_google_genai\chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Chain type: StuffDocumentsChain
Run time: 45.50914509996073
Checkpoint saved at chapter 17


In [42]:
from langchain_community.embeddings import HuggingFaceEmbeddings

def encode_book(path, chunk_size=1000, chunk_overlap=200):
    """
    Encodes a PDF book into a FAISS vector store using HuggingFace embeddings.

    Args:
        path (str): The path to the PDF file.
        chunk_size (int): The desired size of each text chunk.
        chunk_overlap (int): The amount of overlap between consecutive chunks.

    Returns:
        FAISS: A FAISS vector store containing the encoded book content.
    """

    # 1. Load the PDF document using PyPDFLoader
    loader = PyPDFLoader(path)
    documents = loader.load()

    # 2. Split the document into chunks for embedding
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    texts = text_splitter.split_documents(documents)

    # 3. Clean up the text chunks (replace unwanted characters)
    cleaned_texts = replace_t_with_space(texts)

    # 4. Create HuggingFace embeddings and encode the cleaned text chunks into a FAISS vector store
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(cleaned_texts, embeddings)

    # 5. Return the vector store
    return vectorstore


In [43]:
def encode_chapter_summaries(chapter_summaries):
    """
    Encodes a list of chapter summaries into a FAISS vector store using HuggingFace embeddings.

    Args:
        chapter_summaries (list): A list of Document objects representing the chapter summaries.

    Returns:
        FAISS: A FAISS vector store containing the encoded chapter summaries.
    """
    # Create HuggingFace embeddings instance
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    
    # Encode the chapter summaries into a FAISS vector store
    chapter_summaries_vectorstore = FAISS.from_documents(chapter_summaries, embeddings)
    
    # Return the vector store
    return chapter_summaries_vectorstore

In [44]:
def encode_quotes(book_quotes_list):
    """
    Encodes a list of book quotes into a FAISS vector store using HuggingFace embeddings.

    Args:
        book_quotes_list (list): A list of Document objects, each representing a quote from the book.

    Returns:
        FAISS: A FAISS vector store containing the encoded book quotes.
    """
    # Create HuggingFace embeddings instance
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    
    # Encode the book quotes into a FAISS vector store
    quotes_vectorstore = FAISS.from_documents(book_quotes_list, embeddings)
    
    # Return the vector store
    return quotes_vectorstore

In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings
# --- Create or Load Vector Stores for Book Chunks, Chapter Summaries, and Book Quotes ---

# Check if the vector stores already exist on disk
if (
    os.path.exists("chunks_vector_store") and
    os.path.exists("chapter_summaries_vector_store") and
    os.path.exists("book_quotes_vectorstore")
):
    # If vector stores exist, load them using HuggingFace embeddings
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    chunks_vector_store = FAISS.load_local(
        "chunks_vector_store", embeddings, allow_dangerous_deserialization=True
    )
    chapter_summaries_vector_store = FAISS.load_local(
        "chapter_summaries_vector_store", embeddings, allow_dangerous_deserialization=True
    )
    book_quotes_vectorstore = FAISS.load_local(
        "book_quotes_vectorstore", embeddings, allow_dangerous_deserialization=True
    )
else:
    print("not")
    # If vector stores do not exist, encode and save them

    # 1. Encode the book into a vector store of chunks
    chunks_vector_store = encode_book(hp_pdf_path, chunk_size=1000, chunk_overlap=200)

    # 2. Encode the chapter summaries into a vector store
    chapter_summaries_vector_store = encode_chapter_summaries(chapter_summaries)

    # 3. Encode the book quotes into a vector store
    book_quotes_vectorstore = encode_quotes(book_quotes_list)

    # 4. Save the vector stores to disk for future use
    chunks_vector_store.save_local("chunks_vector_store")
    chapter_summaries_vector_store.save_local("chapter_summaries_vector_store")
    book_quotes_vectorstore.save_local("book_quotes_vectorstore")


C:\Users\NAFEES J\AppData\Local\Temp\ipykernel_22152\3118181819.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [5]:
# --- Create Query Retrievers from Vector Stores ---

# The following retrievers are used to fetch relevant documents from the vector stores
# based on a query. The number of results returned can be controlled via the 'k' parameter.

# Retriever for book chunks (returns the top 1 most relevant chunk)
chunks_query_retriever = chunks_vector_store.as_retriever(search_kwargs={"k": 1})

# Retriever for chapter summaries (returns the top 1 most relevant summary)
chapter_summaries_query_retriever = chapter_summaries_vector_store.as_retriever(search_kwargs={"k": 1})

# Retriever for book quotes (returns the top 10 most relevant quotes)
book_quotes_query_retriever = book_quotes_vectorstore.as_retriever(search_kwargs={"k": 10})

In [8]:
import importlib
import helper_functions
importlib.reload(helper_functions)
def retrieve_context_per_question(state):
    """
    Retrieves relevant context for a given question by aggregating content from:
    - Book chunks
    - Chapter summaries
    - Book quotes

    Args:
        state (dict): A dictionary containing the question to answer, with key "question".

    Returns:
        dict: A dictionary with keys:
            - "context": Aggregated context string from all sources.
            - "question": The original question.
    """
    question = state["question"]

    # Retrieve relevant book chunks
    print("Retrieving relevant chunks...")
    docs = chunks_query_retriever.get_relevant_documents(question)
    context = " ".join(doc.page_content for doc in docs)

    # Retrieve relevant chapter summaries
    print("Retrieving relevant chapter summaries...")
    docs_summaries = chapter_summaries_query_retriever.get_relevant_documents(question)
    context_summaries = " ".join(
        f"{doc.page_content} (Chapter {doc.metadata['chapter']})" for doc in docs_summaries
    )

    # Retrieve relevant book quotes
    print("Retrieving relevant book quotes...")
    docs_book_quotes = book_quotes_query_retriever.get_relevant_documents(question)
    book_quotes = " ".join(doc.page_content for doc in docs_book_quotes)
    # Aggregate all contexts and escape problematic characters
    all_contexts = context + context_summaries + book_quotes
    all_contexts =helper_functions.escape_quotes(all_contexts)

    return {"context": all_contexts, "question": question}

In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI
importlib.reload(helper_functions)
# --- Prompt template ---
keep_only_relevant_content_prompt_template = """
You receive a query: {query} and retrieved documents: {retrieved_documents} from a vector store.
You need to filter out all the non relevant information that doesn't supply important information regarding the {query}.
Your goal is just to filter out the non relevant information.
You can remove parts of sentences that are not relevant to the query or remove whole sentences that are not relevant to the query.
DO NOT ADD ANY NEW INFORMATION THAT IS NOT IN THE RETRIEVED DOCUMENTS.
Output the filtered relevant content.
"""

# --- Output schema ---
class KeepRelevantContent(BaseModel):
    relevant_content: str = Field(
        description="The relevant content from the retrieved documents that is relevant to the query."
    )

# --- Prompt + chain ---
keep_only_relevant_content_prompt = PromptTemplate(
    template=keep_only_relevant_content_prompt_template,
    input_variables=["query", "retrieved_documents"],
)

keep_only_relevant_content_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0,
    convert_system_message_to_human=True
)

keep_only_relevant_content_chain = (
    keep_only_relevant_content_prompt
    | keep_only_relevant_content_llm.with_structured_output(KeepRelevantContent)
)

# --- Filtering function ---
def keep_only_relevant_content(state):
    """
    Filters and keeps only the relevant content from the retrieved documents that is relevant to the query.
    """
    question = state["question"]
    context = state["context"]

    input_data = {
        "query": question,
        "retrieved_documents": context
    }

    print("Keeping only the relevant content...")
    pprint("--------------------")
    output = keep_only_relevant_content_chain.invoke(input_data)
    relevant_content = output.relevant_content
    relevant_content = "".join(relevant_content)

    # Escape quotes if needed (define escape_quotes function elsewhere if used)
    relevant_content = helper_functions.escape_quotes(relevant_content)

    return {
        "relevant_context": relevant_content,
        "context": context,
        "question": question
    }

In [13]:
# --- LLM-based Function to Rewrite a Question for Better Vectorstore Retrieval ---

class RewriteQuestion(BaseModel):
    """
    Output schema for the rewritten question.
    """
    rewritten_question: str = Field(
        description="The improved question optimized for vectorstore retrieval."
    )
    explanation: str = Field(
        description="The explanation of the rewritten question."
    )

# Create a JSON parser for the output schema
rewrite_question_string_parser = JsonOutputParser(pydantic_object=RewriteQuestion)

# Initialize the LLM for rewriting questions
rewrite_llm = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",
    groq_api_key=groq_api_key,
    max_tokens=4000
)

# Define the prompt template for question rewriting
rewrite_prompt_template = """
You are a question re-writer that converts an input question to a better version optimized for vectorstore retrieval.
Analyze the input question {question} and try to reason about the underlying semantic intent / meaning.
{format_instructions}
"""
# Create the prompt object
rewrite_prompt = PromptTemplate(
    template=rewrite_prompt_template,
    input_variables=["question"],
    partial_variables={"format_instructions": rewrite_question_string_parser.get_format_instructions()},
)

# Combine prompt, LLM, and parser into a chain
question_rewriter = rewrite_prompt | rewrite_llm | rewrite_question_string_parser

def rewrite_question(state):
    """
    Rewrites the given question using the LLM to optimize it for vectorstore retrieval.

    Args:
        state (dict): A dictionary containing the question to rewrite, with key "question".

    Returns:
        dict: A dictionary with the rewritten question under the key "question".
    """
    question = state["question"]
    print("Rewriting the question...")
    result = question_rewriter.invoke({"question": question})
    new_question = result["rewritten_question"]
    return {"question": new_question}

In [14]:
gemini_model = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0,
    convert_system_message_to_human=True
)

def generate_answer(state):
    """
    Generates an answer to the given question using Gemini 2.5 based on filtered context.
    """

    # ✅ Step 1: Rewrite the question using Groq LLaMA3
    rewritten_state = rewrite_question(state)
    question = rewritten_state["question"]

    # ✅ Step 2: Retrieve context based on rewritten question
    retrieval_output = retrieve_context_per_question({"question": question})
    context = retrieval_output["context"]

    # ✅ Step 3: Filter relevant content
    filtered_output = keep_only_relevant_content({
        "question": question,
        "context": context
    })
    relevant_context = filtered_output["relevant_context"]

    # ✅ Step 4: Create prompt for Gemini
    prompt = f"""You are a helpful assistant answering questions based on the following context from a book. 
Please check if the answer exists in the context. If yes, answer appropriately. 
If not, reply: "Sorry, the question was out of context."

Context:
{relevant_context}

Question:
{question}

Answer:"""

    # ✅ Step 5: Generate final response
    response = gemini_model.invoke(prompt)

    return {
        "question": question,
        "context": relevant_context,
        "answer": response.content.strip()
    }


In [ ]:
import gradio as gr

# This is your RAG function using Gemini
def rag_interface(question):
    state = {"question": question}
    output = generate_answer(state)
    return output["answer"]

# Launch a Gradio interface
demo = gr.Interface(
    fn=rag_interface,
    inputs=gr.Textbox(lines=2, placeholder="Ask a question about the book..."),
    outputs="text",
    title="📘 Book Q&A with Gemini",
    description="Ask any question about the book. Answers are generated using Gemini and retrieved from book chunks, chapter summaries, and quotes."
)

demo.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Rewriting the question...
Retrieving relevant chunks...


C:\Users\NAFEES J\AppData\Local\Temp\ipykernel_22152\2536062878.py:23: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = chunks_query_retriever.get_relevant_documents(question)


Retrieving relevant chapter summaries...
Retrieving relevant book quotes...
Keeping only the relevant content...
'--------------------'


e:\RAGAS\venv\Lib\site-packages\langchain_google_genai\chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
e:\RAGAS\venv\Lib\site-packages\langchain_google_genai\chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
